#### Life cycle of Machine learning Project

data analisys 

- Understanding the Problem Statement
- Data Collection
- Data Checks to perform
- Exploratory data analysis

modeling development
- Understanding the Problem Statement
- Data collection
- Data Pre-Processing
- Feature Engineering
- Model Training
- Choose best model

model deploying

- structure de code in modular programing
- deploy the model in aws


### 1.0 problem statement

una empresa X, tiene una base de datos de clientes, en sql , la empresa almacena difentes datos de los usuarios
CustomerID	Gender	Age	Annual Income ($)	Spending Score (1-100)	Profession	Work Experience	Family Size
, y quiero crear un programa , que a partir de los datos sea capas de predecir , el spending score, 
el departamento te a asignado, por que eres el nuevo, y quieres, que realices todo el proceso de la creacion del proyecto
desde la obtencion de los datos, su analisis, hasta la creacion del modelo, y su despliegue, quieren que prepares
todo , para que este disponible como , un microservicio, pero no lo desplegaras solo , lo dejaras preparado, para desplegarlo

### 2.0 Data Colection

### librerias a utilizar

In [218]:
from sqlalchemy import create_engine
import pandas as pd
#imputation
from sklearn.impute import SimpleImputer

#preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline

#modelos

from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor 
from sklearn.svm import SVR

#for divide de dataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#metrics 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


### 2.1 collect the data from database

In [166]:
#se supone que esta parte me la da la empresa pero tambien podria pasar que dan desde una api

driver = "ODBC+Driver+17+for+SQL+Server"
server_name = "localhost"
database = "BDdatasets"
UID = "sa"
PWD = "0440"

connection_string = f"mssql+pyodbc://{UID}:{PWD}@{server_name}/{database}?driver={driver}"
engine = create_engine(connection_string)

query = "SELECT * FROM Customers"

df = pd.read_sql_query(query , engine)

### 2.2 show top 5 records

In [167]:
df.head()

,CustomerID,Gender,Age,AnnualIncome,SpendingScore,Profession,WorkExperience,FamilySize
0,1,Male,19,15000.0,39.0,Healthcare,1,4
1,2,Male,21,35000.0,81.0,Engineer,3,3
2,3,Female,20,86000.0,6.0,Engineer,1,1
3,4,Female,23,59000.0,77.0,Lawyer,0,2
4,5,Female,31,38000.0,40.0,Entertainment,2,6


### 2.3 check the dataset info

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CustomerID      2000 non-null   object 
 1   Gender          2000 non-null   object 
 2   Age             2000 non-null   int64  
 3   AnnualIncome    2000 non-null   float64
 4   SpendingScore   2000 non-null   float64
 5   Profession      1965 non-null   object 
 6   WorkExperience  2000 non-null   int64  
 7   FamilySize      2000 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 125.1+ KB


### 3.0 data preprocessing 

### 3.1 Preparing X and y

In [178]:
X = df.drop(columns = [ "CustomerID", "SpendingScore"])
y = df["SpendingScore"]/100

### 3.2 Creating the preprocessor

In [179]:
numerical_features = [feature for feature in X.columns if X[feature].dtype != "O"]
categorical_features = [feature for feature in X.columns if X[feature].dtype =="O"]

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",categorical_transformer , categorical_features), 
        ("StandardScaler",numeric_transformer, numerical_features)
    ]
)


### 3.3 preprocessing the features

In [180]:
X = preprocessor.fit_transform(X)

In [181]:
X.shape

(2000, 16)

### 3.4 dividing the dataset in train tes

In [182]:
X_train , X_test , y_train , y_test = train_test_split( X,y ,test_size = 0.2 )
X_train.shape , X_test.shape

((1600, 16), (400, 16))

### 4.0 model training

### 4.1 creating a funtion for get the metrics

In [183]:
def evaluate_model( true,predicted ):
    mae = mean_absolute_error(true , predicted)
    mse = mean_squared_error(true , predicted)
    return ( mse, mae )

### 4.2 training various models

In [222]:
models = {
            "xgb" :XGBRegressor(n_estimators =  40) , 
            "AdaBoostRegressor" : AdaBoostRegressor(n_estimators =  40) , 
            "SVR": SVR(kernel='rbf', C=1.0)
}

model_and_score_list = []
r2_list = []

for  model_name, model in models.items():
    model.fit(X_train ,y_train)
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    train_mse, train_mae  = evaluate_model(y_train ,y_pred_train )
    test_mse, test_mae = evaluate_model(y_test , y_pred_test)

    print(f"model : {model_name}")
    print("Model perfomance for training set")
    print(f"Mean Square Error {train_mse:.5f}")
    print(f"Mean Absolute Error {train_mae:.5f}")
    #print(f"r2_score {train_r2_score:.5f}")
    print("-"*35)

    print("Model perfomance for test set")
    print(f"Mean Square Error {test_mse:.5f}")
    print(f"Mean Absolute Error {test_mae:.5f}")
    #print(f"r2_score {test_r2_score:.5f}")
    print("="*35)
    print("\n\n")

    model_and_score_list.append([model_name , test_mse, test_mae ])

model : xgb
Model perfomance for training set
Mean Square Error 0.02375
Mean Absolute Error 0.12191
-----------------------------------
Model perfomance for test set
Mean Square Error 0.09527
Mean Absolute Error 0.25772



model : AdaBoostRegressor
Model perfomance for training set
Mean Square Error 0.07582
Mean Absolute Error 0.23562
-----------------------------------
Model perfomance for test set
Mean Square Error 0.07903
Mean Absolute Error 0.23526



model : SVR
Model perfomance for training set
Mean Square Error 0.05931
Mean Absolute Error 0.20017
-----------------------------------
Model perfomance for test set
Mean Square Error 0.09146
Mean Absolute Error 0.25089





### 5.0 choose de best model

In [224]:
pd.DataFrame(model_and_score_list , columns = [ "Name", "MSE" , "MAE"])

,Name,MSE,MAE
0,xgb,0.095270,0.257722
1,AdaBoostRegressor,0.079026,0.235257
2,SVR,0.091459,0.250890


el modelo con mejor rendimiento fue el adaboostregressor